### RDD exercise:

- list the top-10 most frequent words in the "Inferno" document
- find the average number of occurrences of each word in "Inferno"
- list 10 words that occur more frequently than the average (try this on your own!)

#### load text file -- this creates an RDD consisting of strings, one for each line of text

doc: [textFile()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkContext.textFile.html?highlight=textfile#pyspark.SparkContext.textFile)

In [0]:
lines = sc.textFile('/FileStore/tables/Dante_Inferno.txt')


#### split each line into words. 

each worker will operate on a batch of input lines. The workers would run in parallel

doc:
- [map()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.map.html?highlight=rdd%20map#pyspark.RDD.map)
- [flatMap()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.flatMap.html?highlight=rdd%20flatmap#pyspark.RDD.flatMap)

In [0]:
words = lines.flatMap(lambda l: l.split(" "))


In [0]:
## show a few lines
words.take(20)

Out[70]: ['',
 '',
 'LA',
 'DIVINA',
 'COMMEDIA',
 '',
 '',
 'di',
 'Dante',
 'Alighieri',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'INFERNO',
 '',
 '']

#### remove blank and short tokens

doc: [filter()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.filter.html?highlight=rdd%20filter#pyspark.RDD.filter)

In [0]:
filteredWords = words.filter(lambda x:  len(x) > 5)

In [0]:
## show a few lines
filteredWords.take(20)

Out[36]: ['DIVINA',
 'COMMEDIA',
 'Alighieri',
 'INFERNO',
 'Inferno',
 'cammin',
 'nostra',
 'ritrovai',
 'oscura,',
 'diritta',
 'smarrita.',
 'quanto',
 'selvaggia',
 'pensier',
 'rinova',
 'paura!',
 'morte;',
 'trattar',
 'trovai,',
 'l\x92altre']

#### count number of occurrences of each word

this is the `word count` example

doc: [reduceByKey()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.reduceByKey.html?highlight=reducebykey#pyspark.RDD.reduceByKey)

In [0]:
wordCount = filteredWords.map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y)

In [0]:
## show a few results
wordCount.take(20)

Out[38]: [('DIVINA', 1),
 ('Alighieri', 1),
 ('Inferno', 34),
 ('cammin', 2),
 ('nostra', 14),
 ('ritrovai', 1),
 ('diritta', 1),
 ('morte;', 2),
 ('trattar', 1),
 ('trovai,', 1),
 ('l\x92altre', 12),
 ('scorte.', 1),
 ('v\x92intrai,', 1),
 ('verace', 2),
 ('giunto,', 2),
 ('compunto,', 4),
 ('spalle', 6),
 ('altrui', 10),
 ('calle.', 3),
 ('queta,', 3)]

#### list the top-10 most frequent words

we need to sort each pair by frequency. 

we use [sortBy()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.sortBy.html?highlight=sortby) and specify that the sort element is the second in the pair:

- x[0] first element
- x[1] second element

In [0]:
wordCount.sortBy(lambda x: x[1], ascending=False).take(10)

Out[42]: [('quando', 80),
 ('quella', 68),
 ('disse:', 55),
 ('l\x92altro', 44),
 ('perché', 43),
 ('questa', 39),
 ('questo', 38),
 ('Inferno', 34),
 ('maestro', 34),
 ('l\x92altra', 30)]

#### calculate the average number of occurrences of each word

In [0]:
occurrences =  wordCount.map(lambda x: x[1])
occurrences.take(10)

Out[73]: [1, 17, 5, 1, 2, 1, 1, 1, 55, 6]

In [0]:
totalOccurrences = occurrences.reduce(lambda x, y: x+y)
totalOccurrences

Out[74]: 10321

In [0]:
averageOccurrences =  totalOccurrences / occurrences.count()

In [0]:
averageOccurrences

Out[77]: 1.514231220657277